In [ ]:
# !apt-get update 
# !pip3 install pybullet
# !apt-get install -y ffmpeg
# https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#

In [ ]:
import pybullet as p
import time
import numpy as np
import os
import _utils as u
from _utils import geom_shape, xyz, s
import sys
from collections import namedtuple

def coord2xyz(coord):
    return [coord.Y,coord.X,coord.Z]

def structure(J_CollShpInds = None, M_CollShpInds = None,
              stick_from    = None, stick_to      = None, 
              index_J       = None, index_M       = None):
    m2j = ['m','j']; 
    j2m = ['j','m']; 
    m2m = ['m','m']; 
    d2m = ['d','m']; 
    m2d = ['m','d']
    STATIC = [0,0,0]
    if J_CollShpInds is not None:
        J = ROBOT(CollShpInds = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j), 
            Masses = 0.00000001,  Positions   = xyz(m2j,stick_from, stick_to),
            indices = index_J,jntTypes = p.JOINT_REVOLUTE, 
            axis = coord2xyz(u.MOTORS[stick_to].axes), angles = [u.MOTORS[stick_to].min_angle, 
                                                                 u.MOTORS[stick_to].max_angle])
    else:
        J = None
        
    if M_CollShpInds is not None:
        M = ROBOT(CollShpInds = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1), 
            Masses = u.MASSES[stick_to].mass,  Positions   = xyz(j2m,stick_to, stick_to),
            indices = index_M, jntTypes = p.JOINT_PRISMATIC, axis = STATIC , angles = None)
    else:
        M = None
    
    if (J is not None) and (M is not None):
        return J,M
    if (J is not None) and (M is None):
        return J
    if (J is None) and (M is None):
        return M

ROBOT = namedtuple('Body',['CollShpInds', 'Masses', 'Positions', 'indices', 'jntTypes', 'axis', 'angles'])

SPH = p.GEOM_SPHERE
BOX = p.GEOM_BOX
CYL = p.GEOM_CYLINDER
CAP = p.GEOM_CAPSULE

VARIABLE = 0.04
# joint coefficient
j = 0.25

u.init_window()
# b_p = u.coordinates(0,0.9,0)
# base= u.geom_shape(BOX, height = VARIABLE, coeff_P = [1,1,1])
# body_Mass = 0.1
b_p  = u.coordinates(0,0,0)
base = u.geom_shape(BOX, height = VARIABLE, coeff_P = [10,10,0.1])

J_CollShpInds = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_CollShpInds = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)

body_Mass = 20000

m2j = ['m','j']; j2m = ['j','m']; m2m = ['m','m']; d2m = ['d','m']; m2d = ['m','d']
PRISMATIC = p.JOINT_PRISMATIC
REVOLUTE  = p.JOINT_REVOLUTE
STATIC    = [0,0,0]
m         = 0.00000001

In [ ]:
M_Pelvis = ROBOT(
    CollShpInds = geom_shape(SPH, radius = VARIABLE, coeff_R = 1), 
    Masses      = u.MASSES['Pelvis'].mass, 
    Positions   = xyz(d2m,'CMS_Coords','Pelvis', data   = b_p),
    indices     = 0, 
    jntTypes    = PRISMATIC, 
    axis        = STATIC,
    angles      = None)
#===========================================================================================================
# Torso
J_TorsoS, M_TorsoS     = structure(J_CollShpInds, M_CollShpInds,'Pelvis',      'TorsoS',       1,  2)
J_TorsoF, M_TorsoF     = structure(J_CollShpInds, M_CollShpInds,'TorsoS',      'TorsoF',       3,  4)
J_TorsoR, M_TorsoR     = structure(J_CollShpInds, M_CollShpInds,'TorsoF',      'TorsoR',       5,  6)
# Neck and Head
J_Neck,  M_Neck        = structure(J_CollShpInds, M_CollShpInds,'TorsoR',      'Neck',         7,  8)
J_HeadR, M_HeadR       = structure(J_CollShpInds, M_CollShpInds,'Neck',        'HeadR',        9, 10)
J_HeadF, M_HeadF       = structure(J_CollShpInds, M_CollShpInds,'HeadR',       'HeadF',       11, 12)
# Right Arm
R_J_ShldrF, R_M_ShldrF = structure(J_CollShpInds, M_CollShpInds,'TorsoR',      'R_ShoulderF',  7, 14)
R_J_ShldrS, R_M_ShldrS = structure(J_CollShpInds, M_CollShpInds,'R_ShoulderF', 'R_ShoulderS', 15, 16)
R_J_ElbowR, R_M_ElbowR = structure(J_CollShpInds, M_CollShpInds,'R_ShoulderS', 'R_ElbowR',    17, 18)
R_J_Elbow,  R_M_Elbow  = structure(J_CollShpInds, M_CollShpInds,'R_ElbowR',    'R_Elbow',     19, 20)
# Left Arm
L_J_ShldrF, L_M_ShldrF = structure(J_CollShpInds, M_CollShpInds,'TorsoR',      'L_ShoulderF',  7, 22)
L_J_ShldrS, L_M_ShldrS = structure(J_CollShpInds, M_CollShpInds,'L_ShoulderF', 'L_ShoulderS', 23, 24)
L_J_ElbowR, L_M_ElbowR = structure(J_CollShpInds, M_CollShpInds,'L_ShoulderS', 'L_ElbowR',    25, 26)
L_J_Elbow,  L_M_Elbow  = structure(J_CollShpInds, M_CollShpInds,'L_ElbowR',    'L_Elbow',     27, 28)
# Right Leg
R_J_HipF, R_M_HipF     = structure(J_CollShpInds, M_CollShpInds,'Pelvis',      'R_HipF',       1, 30)
R_J_HipS, R_M_HipS     = structure(J_CollShpInds, M_CollShpInds,'R_HipF',      'R_HipS',      31, 32)
R_J_HipR, R_M_HipR     = structure(J_CollShpInds, M_CollShpInds,'R_HipS',      'R_HipR',      33, 34)
R_J_Knee, R_M_Knee     = structure(J_CollShpInds, M_CollShpInds,'R_HipR',      'R_Knee',      35, 36)
R_J_AnkleF, R_M_AnkleF = structure(J_CollShpInds, M_CollShpInds,'R_Knee',      'R_AnkleF',    37, 38)
R_J_AnkleS, R_M_AnkleS = structure(J_CollShpInds, M_CollShpInds,'R_AnkleF',    'R_AnkleS',    39, 40)
# Left Leg
L_J_HipF,   L_M_HipF   = structure(J_CollShpInds, M_CollShpInds,'Pelvis',      'L_HipF',       1, 43)
L_J_HipS,   L_M_HipS   = structure(J_CollShpInds, M_CollShpInds,'L_HipF',      'L_HipS',      44, 45)
L_J_HipR,   L_M_HipR   = structure(J_CollShpInds, M_CollShpInds,'L_HipS',      'L_HipR',      46, 47)
L_J_Knee,   L_M_Knee   = structure(J_CollShpInds, M_CollShpInds,'L_HipR',      'L_Knee',      48, 49)
L_J_AnkleF, L_M_AnkleF = structure(J_CollShpInds, M_CollShpInds,'L_Knee',      'L_AnkleF',    50, 51)
L_J_AnkleS, L_M_AnkleS = structure(J_CollShpInds, M_CollShpInds,'L_AnkleF',    'L_AnkleS',    52, 53)
R_M_Foot = ROBOT(
    CollShpInds = geom_shape(BOX, height = VARIABLE, coeff_P = [1,3,0.25]), 
    Masses      = u.MASSES['R_Foot'].mass, 
    Positions   = xyz(m2m,'R_AnkleS','R_Foot'),
    indices     = 41, 
    jntTypes    = PRISMATIC, 
    axis        = STATIC,
    angles      = None)
L_M_Foot = ROBOT(
    CollShpInds = geom_shape(BOX, height = VARIABLE, coeff_P = [1,3,0.25]), 
    Masses      = u.MASSES['L_Foot'].mass, 
    Positions   = xyz(m2m,'L_AnkleS','L_Foot'),
    indices     = 54, 
    jntTypes    = PRISMATIC, 
    axis        = STATIC,
    angles      = None)

cfgs = [ 
  M_Pelvis, 
  J_TorsoS,   M_TorsoS,   J_TorsoF,   M_TorsoF,   J_TorsoR,   M_TorsoR,
  J_Neck,     M_Neck,     J_HeadR,    M_HeadR,    J_HeadF,    M_HeadF,
R_J_ShldrF, R_M_ShldrF, R_J_ShldrS, R_M_ShldrS, R_J_ElbowR, R_M_ElbowR, R_J_Elbow, R_M_Elbow,
L_J_ShldrF, L_M_ShldrF, L_J_ShldrS, L_M_ShldrS, L_J_ElbowR, L_M_ElbowR, L_J_Elbow, L_M_Elbow,
R_J_HipF,   R_M_HipF,   R_J_HipS,   R_M_HipS,   R_J_HipR,   R_M_HipR,   R_J_Knee,  R_M_Knee, 
R_J_AnkleF, R_M_AnkleF, R_J_AnkleS, R_M_AnkleS, R_M_Foot,   
L_J_HipF,   L_M_HipF,   L_J_HipS,   L_M_HipS,   L_J_HipR,   L_M_HipR,   L_J_Knee,  L_M_Knee,
L_J_AnkleF, L_M_AnkleF, L_J_AnkleS, L_M_AnkleS, L_M_Foot]

linkCollisionShapeIndices = [p[0] for p in cfgs]
link_Masses               = [p[1] for p in cfgs]
linkPositions             = [p[2] for p in cfgs]
indices                   = [p[3] for p in cfgs]
jointTypes                = [p[4] for p in cfgs]
axis                      = [p[5] for p in cfgs]
angles                    = [p[6] for p in cfgs] 

In [ ]:
Torso     = [1,3,5] 
NandH     = [7,9,11] 
Right_arm = [13,15,17,19]
Left_arm  = [21,23,25,27]
Right_leg = [29,31,33,35,37,39]
Left_leg  = [42,44,46,48,50,52]

# BLOCK Init
top = Torso + NandH + Right_arm + Left_arm
bot = Right_leg + Left_leg

# default params 
motorIndexes = top + bot
linkValues   = [0] * len(motorIndexes)

visualShapeId   = -1
nlnk            = len(link_Masses)
LVSIndices      = [-1] * nlnk 
LOrientations   = [[0,0,0,1]] * nlnk
LIFPositions    = [[0,0,0]]   * nlnk
LIFOrientations = [[0,0,0,1]] * nlnk
#Drop the body in the scene at the following body coordinates
baseOrientation = [0,0,0,1]
robot_positions = [[0,0,1]] # b_p.Z 

for basePosition in robot_positions:
    robot = p.createMultiBody(
        baseMass                      = body_Mass,
        baseCollisionShapeIndex       = base,
        baseVisualShapeIndex          = visualShapeId,
        basePosition                  = basePosition,
        baseOrientation               = baseOrientation,
        
        linkVisualShapeIndices        = LVSIndices,
        linkOrientations              = LOrientations,
        linkInertialFramePositions    = LIFPositions,
        linkInertialFrameOrientations = LIFOrientations,
        
        linkMasses                    = link_Masses,
        linkCollisionShapeIndices     = linkCollisionShapeIndices,
        linkPositions                 = linkPositions,
        linkParentIndices             = indices,
        linkJointTypes                = jointTypes,
        linkJointAxis                 = axis
    )
    u.resetCoordinates(robot, motorIndexes, linkValues)

# Add earth like gravity
p.setGravity(0,0,-9.81)
p.setRealTimeSimulation(1)

cameraDistance = 5.0
cameraYaw      = 180
cameraPitch    = -27.60

u.resetCamera(
    cameraDistance = cameraDistance,
    cameraYaw      = cameraYaw,
    cameraPitch    = cameraPitch,
)

angle_HeadF       = 0; angle_HeadR       = 0;       angle_neck   = 0;
angle_TorsoS      = 0; angle_TorsoF      = 0;       angle_TorsoR = 0;
angle_R_ShoulderF = 0; angle_L_ShoulderF = 0
angle_R_ShoulderS = 0; angle_L_ShoulderS = 0
angle_R_ElbowR    = 0; angle_L_ElbowR    = 0
angle_R_Elbow     = 0; angle_L_Elbow     = 0
angle_R_HipF      = 0; angle_L_HipF      = 0
angle_R_HipS      = 0; angle_L_HipS      = 0
angle_R_HipR      = 0; angle_L_HipR      = 0
angle_R_Knee      = 0; angle_L_Knee      = 0
angle_R_AnkleF    = 0; angle_L_AnkleF    = 0
angle_R_AnkleS    = 0; angle_L_AnkleS    = 0

In [ ]:
TorsoS      = [ -15,  15]   
TorsoF      = [ -12,  40] 
TorsoR      = [ -95,  95]
Neck        = [ -10,  40]   
HeadR       = [ -80,  80] 
HeadF       = [ -60,  20]
ShoulderF   = [-150,  90] 
L_ShoulderS = [   0,-150] 
R_ShoulderS = [ 150,   0]
ElbwR       = [-110, 110] 
Elbw        = [ 120,   5]
__HipF      = [ -90,  35]   
R_HipS      = [ -90,   2] 
L_HipS      = [ -12,  90] 
__HipR      = [ -90,  90]
__Knee      = [   0, 110]   
__AnkleF    = [ -40,  25] 
__AnkleS    = [ -30,  30]

In [ ]:
from MAIN_LOOP.printer import printer
from MAIN_LOOP.options import options
from MAIN_LOOP.body_mv import move_head, move_torso, move_arms, move_legs

# angleFHead, angleRHead, angle_neck
main_flag = True
while main_flag:
    msg = 'Main: Options(O), BodyMove(M), Info(?), Quit(Q)'
    printer(msg)
    keys = p.getKeyboardEvents()
    if keys.get(111): # o - options
        result = options(cameraDistance, cameraYaw, cameraPitch)
        cameraDistance, cameraYaw, cameraPitch = result
    if keys.get(109): # m - move body paths
        move_flag = True
        while move_flag:
            printer('Move: Head(H), Torso(T), Arms(A), Legs(L), Info(?). Quit(1)')
            move_loop = p.getKeyboardEvents()
            if move_loop.get(104):# H - Head
                angles      = [angle_neck, angle_HeadR, angle_HeadF]
                minmaxNandH = [Neck, HeadR, HeadF]
                angles      = move_head(robot, NandH, angles, minmaxNandH)
            if move_loop.get(116):# T - Torso
                angles      = [angle_TorsoS, angle_TorsoF, angle_TorsoR]
                minmaxTorso = [TorsoS,TorsoF,TorsoR]
                angles      = move_torso(robot, Torso, angles, minmaxTorso)
            if move_loop.get(97): # A - Arms
                angles      = [
                    angle_R_ShoulderF, angle_L_ShoulderF,
                    angle_R_ShoulderS, angle_L_ShoulderS,
                    angle_R_ElbowR,    angle_L_ElbowR,
                    angle_R_Elbow,     angle_L_Elbow,]
                minmaxArms  = [ShoulderF, [R_ShoulderS, L_ShoulderS], ElbwR, Elbw]
                Arms        = ShldrF + ShldrS + ElbowR + Elbow
                angles      = move_arms(robot, Arms, angles, minmaxArms)
            if move_loop.get(108):# L - Legs
                angles      = [
                    angle_R_HipF,   angle_L_HipF,
                    angle_R_HipS,   angle_L_HipS,
                    angle_R_HipR,   angle_L_HipR,
                    angle_R_Knee,   angle_L_Knee,
                    angle_R_AnkleF, angle_L_AnkleF,
                    angle_R_AnkleS, angle_L_AnkleS,]
                minmaxLegs = [__HipF, [R_HipS, L_HipS], __HipR, __Knee, __AnkleF, __AnkleS]
                Legs       = HipF + HipS + HipR + Knee + AnkleF + AnkleS
                angles     = move_legs(robot, Legs, angles, minmaxLegs)
            if move_loop.get(49): # Quit MV Block
                move_flag = False
            if move_loop.get(47):
                msg = 'MV Block: press "1" to go "Main"'
                printer(msg)
            time.sleep(0.3)
    if keys.get(47):
        printer(msg)
    if keys.get(113):
        printer('Quit Main...')
        main_flag = False
    time.sleep(0.1)
p.disconnect()